## INITIALIZE REQUIREMENTS

### Load Dependencies

In [1]:
%matplotlib inline

import os
import glob
import warnings 
import datacube
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
warnings.filterwarnings("ignore")

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage.filters import threshold_minimum
from datacube.utils.geometry import Geometry

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.areaofinterest import define_area

### Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
dc = datacube.Datacube(app="Radar_water_detection")

## CUSTOMIZE DATA
In the next three cells define the flood dates, threshold value and grid vector file

### Define Flooding Dates

In [3]:
# DEFINE PERIODS
pre_flood = ['2024-04-15', '2024-05-15', '2024-06-15', '2023-07-15'] # 4 MONTHS PRIOR
flood = ['2024-07-26', '2024-08-26', '2024-09-26', '2024-10-15'] # 4 MONTHS DURING
# post_flood = ['2024-09-10', '2024-10-15']

### Define Threshold

In [4]:
# threshold_vh = -35

### Upload file and calculate centroids

Upload grid file with grid of ~10km x 10km or 0.1 degree. Here grid file of Lake Chad is used with 0.1 degree grid size

In [5]:
# UPLOAD FILE
grid = gpd.read_file("input/Lake Chad.geojson")

# Calculate centroids and store in centroid list c[]. The array c[] will be used to loop all grid cells
c = [] # INIT EMPTY GRID CENTROID
g = grid.centroid # STORE ALL GRID CENTROIDS in g
for i in g:
    c.append([round(i.x, 5), round(i.y, 5)]) # EXTRACT x and y FROM POINT(x,y) AND APPEND TO c[]

# AOI MOSAIC COVERING LAKE CHAD
aoi_m = []
for i in c:
    aoi_m.append(define_area(i[1], i[0], buffer=0.05))

## FUNCTION DEFINITIONS

### SPECKLE FILTER FUNCTION

In [14]:
#defining a function to apply lee filtering on S1 image 
def lee_filter(da, size):
    """
    Apply lee filter of specified window size.
    Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python

    """
    img = da.values
    img_mean = uniform_filter(img, size)
    img_sqr_mean = uniform_filter(img**2, size)
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    
    return img_output

### CLASSIFIER FUNCTION

In [15]:
def S1_water_classifier(da, threshold):
    water_data_array = da < threshold
    return water_data_array.to_dataset(name="s1_water")

## Main For Loop Interation

In [16]:
grid_val = 1

for aoi in aoi_m:
    print("\n\n"+ '\033[32m' + "PROCESSING GRID CELL {}/{}".format(grid_val, len(aoi_m)) + '\033[0m')
    geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
    geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)

    # Get the latitude and longitude range of the geopolygon
    lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
    lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

    #timeframe
    timerange = ('2024-01-01', '2024-10-01')

    # LOAD SENTINEL-1 DATA
    S1 = load_ard(dc=dc,
                products=["s1_rtc"],
                measurements=['vv', 'vh'],
                y=lat_range,
                x=lon_range,
                time=timerange,
                output_crs = "EPSG:6933",
                resolution = (-20,20),
                group_by="solar_day",
                dtype='native'
                )
    
    # INIT TIMESTEPS 
    timesteps = [2,4,6,9,11]

    # VH/VV is a potentially useful third feature after VV and VH 
    S1['vh/vv'] = S1.vh/S1.vv  

    # MEDIAN VALUES FOR SIMILAR RANGE FOR VISUALIZATION
    med_s1 = S1[['vv','vh','vh/vv']].median()  

    # The lee filter above doesn't handle null values
    # We therefore set null values to 0 before applying the filter
    valid = np.isfinite(S1)
    S1 = S1.where(valid, 0)

    # Create a new entry in dataset corresponding to filtered VV and VH data
    S1["filtered_vv"] = S1.vv.groupby("time").apply(lee_filter, size=7)
    S1["filtered_vh"] = S1.vh.groupby("time").apply(lee_filter, size=7)

    # Null pixels should remain null
    S1['filtered_vv'] = S1.filtered_vv.where(valid.vv)
    S1['filtered_vh'] = S1.filtered_vh.where(valid.vh)   

    # Convert the digital numbers to dB
    S1['filtered_vv'] = 10 * np.log10(S1.filtered_vv)
    S1['filtered_vh'] = 10 * np.log10(S1.filtered_vh)

    threshold_vh = threshold_minimum(S1.filtered_vh.values)
    print(threshold_vh)

    S1['water'] = S1_water_classifier(S1.filtered_vh, threshold_vh).s1_water

    # CREATE FLOODING OUTPUTS
    S1['water_APR_JUL_24'] = S1.water.sel(time = pre_flood, method = 'nearest').mean(dim = 'time')
    S1['water_JUL_OCT_24'] = S1.water.sel(time = flood, method = 'nearest').mean(dim = 'time')
    # S1['water_SEP_OCT_24'] = S1.water.sel(time = post_flood, method = 'nearest').mean(dim = 'time')

    # EXPORT TO RASTERS
    out_ras_name = 'CELL_' + str(grid_val) + '_PRE_FLOOD.tif'
    src_out = 'output/preflood/' + out_ras_name
    S1.water_APR_JUL_24.rio.to_raster(src_out)

    out_ras_name = 'CELL_' + str(grid_val) + '_FLOOD.tif'
    src_out = 'output/flood/' + out_ras_name
    S1.water_JUL_OCT_24.rio.to_raster(src_out)

    # out_ras_name = 'CELL_' + str(grid_val) + '_POST_FLOOD_Mean.tif'
    # src_out = 'output/postflood/' + out_ras_name
    # S1.water_SEP_OCT_24.rio.to_raster(src_out)

    grid_val += 1





PROCESSING GRID CELL 1/5
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 42 time steps
-38.336662


PROCESSING GRID CELL 2/5
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 42 time steps
-25.997452


PROCESSING GRID CELL 3/5
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 42 time steps
-25.845648


PROCESSING GRID CELL 4/5
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 42 time steps
-25.63038


PROCESSING GRID CELL 5/5
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 42 time steps
-21.168266


## Write Outputs To Disc

In [57]:
dirs = ['flood', 'preflood', 'postflood']

for dir in dirs:
    loc = "output/" + dir
    out = "output/{}/Merged_{}.tif".format(dir, dir)
    extension = "*.tif"
    q = os.path.join(loc, extension)
    files = glob.glob(q)

    r =[]
    for f in files:
        s = rasterio.open(f)
        r.append(s)
    if len(r)>0:
        mosaic, out_trans = merge(r)
        out_meta = s.meta.copy()
        out_meta.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans
                    })
        with rasterio.open(out, "w", **out_meta) as dest:
            dest.write(mosaic)